In [6]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


class ItemRecommendation:
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path)
        self.data['processed_name'] = self.data['name'].str.lower()

        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(self.data['processed_name'])

    def find_similar_items(self, item_id, top_n=20):
        if item_id not in self.data['id'].values:
            print(f"Item ID {item_id} not found in dataset.")
            return []

        item_index = self.data[self.data['id'] == item_id].index[0]

        # Get the TF-IDF vector for the item
        item_vector = self.tfidf_matrix[item_index]

        # Compute cosine similarities
        cosine_similarities = cosine_similarity(item_vector, self.tfidf_matrix).flatten()

        # Get indices of the top similar items, excluding the item itself
        similar_indices = cosine_similarities.argsort()[-top_n-1:-1][::-1]

        # Fetch item details for similar items without similarity_score
        similar_items = self.data.iloc[similar_indices][['name', 'main_category', 'sub_category']]

        # Convert the DataFrame to a list of dictionaries
        similar_items_list = similar_items.to_dict(orient='records')

        return similar_items_list


    def save(self, filepath):
        """Save the model to a file."""
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(filepath):
        """Load the model from a file."""
        with open(filepath, 'rb') as f:
            return pickle.load(f)


# Example Usage
if __name__ == "__main__":
    # Initialize and save the model
    item_model = ItemRecommendation(data_path="test_data.csv")
    item_model.save('D:/RecommendModel/Saved_model/item_recommendation_model.pkl')

    # Load the model

    # Find similar items
    item_id = "tvc119790"
    similar_items = item_model.find_similar_items(item_id, top_n=20)
    print("Similar items for Item ID:", item_id)
    print(similar_items)


Similar items for Item ID: tvc119790
[{'name': 'FND Camera Lens UV Filter 49MM -Super Slim UV Filter Aluminum Frame (49 MM)', 'main_category': 'tv, audio & cameras', 'sub_category': 'Camera Accessories'}, {'name': 'Shomex UV Protection Lens Filter (72 mm)', 'main_category': 'tv, audio & cameras', 'sub_category': 'Cameras'}, {'name': 'Shomex UV Protection Lens Filter (72 mm)', 'main_category': 'tv, audio & cameras', 'sub_category': 'Camera Accessories'}, {'name': 'Hanumex UV Protection Lens Filter 55 mm', 'main_category': 'tv, audio & cameras', 'sub_category': 'Camera Accessories'}, {'name': 'VIBLITZ® Ultra Slim UV Filter Protection Slim Frame (77 MM)', 'main_category': 'tv, audio & cameras', 'sub_category': 'Cameras'}, {'name': 'AmazonBasics UV Protection Lens Filter - 77 mm', 'main_category': 'tv, audio & cameras', 'sub_category': 'Camera Accessories'}, {'name': 'Hanumex UV Protection Lens Filter 77 mm', 'main_category': 'tv, audio & cameras', 'sub_category': 'Camera Accessories'}, {'